In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import random
from operator import add

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [7]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param K {I} >=0;   # capacity of machine 

var X {I,J} >=0 ;  # Decision var 1

maximize Benefit: sum {i in I, j in J} B[j] * X[i,j];

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= K[i];
s.t. ProdTime {        j in J}: sum {i in I}        X[i,j] <= 1;
s.t. Arrange  {i in I, j in J}:                     X[i,j] >= 0;

''')

## Step 5: Define the initial data

In [8]:
# batch 要做成:
# [ num_machine, num_job, cost, benefit, capacity ]

it_n = 100

def batch (num_machine, num_job, relation_bc, capacity):
    
    m = num_machine
    n = num_job
    
    list_c1 = []    # 內含 100組 n個 job的 cost 組合
    list_b1 = []    # 內含 100組 n個 job的 bnft 組合
    list_k1 = []    # 內含 100組 由 n個 job 加總算出的 k
    
    for i in range(it_n):
        
        list_c2 = []   # 內含 n個 job的 cost
        list_b2 = []   # 內含 n個 job的 bnft
        
        for j in range(n):
            c = int(random.randint(0,50))
            list_c2.append(c)

            if   relation_bc == 'L':
                b = c

            elif relation_bc == 'X':
                b = c**2

            elif relation_bc == 'A':
                b = c**(1/2)

            elif relation_bc == 'R':
                b = random.randint(0,50)  
            list_b2.append(b)

        if   capacity =='N':
            k = 10000 # no capacity

        elif capacity =='L':
            k = sum(list_c2) / m

        elif capacity =='T':
            k = sum(list_c2) * (0.75) / m
    
        list_c1.append(list_c2)
        list_b1.append(list_b2)
        list_k1.append(k)
    
    return  m, n, list_c1, list_b1, list_k1

In [9]:
# Test OK! 不要在正式 run 100組時  print，會很慢 >"<

instance = batch( 5, 20, 'A', 'T' )
print(instance[2])
print(instance[3])


[[33, 27, 40, 46, 22, 14, 36, 48, 16, 39, 50, 35, 10, 50, 42, 13, 27, 38, 32, 49]]
[[5.744562646538029, 5.196152422706632, 6.324555320336759, 6.782329983125268, 4.69041575982343, 3.7416573867739413, 6.0, 6.928203230275509, 4.0, 6.244997998398398, 7.0710678118654755, 5.916079783099616, 3.1622776601683795, 7.0710678118654755, 6.48074069840786, 3.605551275463989, 5.196152422706632, 6.164414002968976, 5.656854249492381, 7.0]]


In [9]:
machine_job  = [(5,50),(15,50),(15,500),(50,500)]
capacity     = ['L','T','N'] 
relation_bc  = ['L','A','X','R'] 

## Step 6: Model Build-up & Solution

In [10]:
# 迭代 instance

TotalBenefit_ijk_sn_1 = []
TotalBenefit_ijk_sn_2 = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]

    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for k in capacity:    
        for j in relation_bc:

            instance = batch( i[0], i[1], j, k )
            #print(instance)  
            
            TotalBenefit_1 = []
            TotalBenefit_2 = []
            
            for p in range(it_n):      
                
                # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均
                
                # amplpy DataFrame
                ampl.setData(DataFrame(
                index=[('J', job)], 
                columns=[
                    ('B', instance[3][p]), 
                    ('C', instance[2][p])
                ]
                ))

                # Pandas DataFrame
                df = pd.DataFrame({
                    'K': [ instance[4][p] ]*num_machine
                }, 
                    index = machine
                )
                ampl.setData(DataFrame.fromPandas(df))

                # Solve the problem
                ampl.solve()

                '''
                # Create a DataFrame with Decision variables
                Var = ampl.getVariable('X').getValues().toPandas()
                print(Var)
                '''

                # Display the objective value
                TotalBnft_1 = ampl.getObjective('Benefit')
                TotalBnft_1 = TotalBnft_1.value()
                TotalBenefit_1 = np.append(TotalBenefit_1, TotalBnft_1) # numpy.ndarray
                
                
                # ====== Model 2 ======= : 一組一組用 CHBF算，每種senario要算 100組 再平均
                
                # 將 job 按照 cost (workload) 排序

                list_c = instance[2][p]
                list_b = instance[3][p]

                list_cb = list(zip(list_c, list_b))        # [3,5]  [8,1]  -> [(3,8),(5,1)]
                s_cb    = sorted(list_cb, reverse = True)  # [(3,8),(5,1)] -> [(5,1),(3,8)]
                o_cb    = list(zip(*s_cb))                 # [(5,1),(3,8)] -> [(5,3),(1,8)]

                list_c  = list(o_cb[0])                    # [5,3]
                list_b  = list(o_cb[1])                    # [1,8]

                mach = []                 # machine
                ca = instance[4][p]

                for r in range(num_machine):

                    mach.append([])    # machine set中安排一台台 machine 被 append 進來
                    mach[r].append(0)  # 初始的 benefit和 是 0
                    mach[r].append(ca) # 初始的 capacity  是 ca

                TotalBnft_2 = 0             
                for s in range(num_job):

                    mach = sorted(mach)                           # 按照 benefit和 由小而大排序
                    # print(machine)

                    for r in range(num_machine):                  # 由最小 benefit和 的先開始
                        
                        if list_c[s] <= mach[r][1]:               # machine r 還有 capacity 餘裕
                            
                            #print(mach[r])
                            mach[r][1] -= list_c[s]
                            mach[r][0] += list_b[s]
                            #print(mach[r])
 
                            break
    
                for r in range(num_machine):                  
                    TotalBnft_2 += mach[r][0]                           # 將所有 machine 的 benefit和加起來成為一組
                    #print(mach[r][0])
                    
                TotalBenefit_2 = np.append(TotalBenefit_2, TotalBnft_2) # 將 100組全 append在一起 
                #print(TotalBenefit_2)
                
                
            # ====== Model 1 Objective value: ====== #
            
            list_M1           = list(TotalBenefit_1)
            TotalBenefit_100_1 = sum(list_M1) / len(list_M1)
            TotalBenefit_ijk_sn_1.append(TotalBenefit_100_1)   # i*j*k 種 senario 迭代
            
            # ====== Model 2 Objective value: ====== #
            
            list_M2           = list(TotalBenefit_2)
            TotalBenefit_100_2 = sum(list_M2) / len(list_M2)
            TotalBenefit_ijk_sn_2.append(TotalBenefit_100_2)  # i*j*k 種 senario 迭代
            
            print('=========================' + str(i) + str(j) + str(k) + '=========================') # 分隔           
            
print(TotalBenefit_ijk_sn_1) # list
print(TotalBenefit_ijk_sn_2) # list

TotalBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, TotalBenefit_ijk_sn_2, TotalBenefit_ijk_sn_1))
TotalBenefit_ijk_sn_ratio

CPLEX 12.8.0.0: optimal solution; objective 1086
87 dual simplex iterations (5 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1312
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1397
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1267
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1186
7 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1345
4 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1278
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1347
8 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1112
7 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1138
6 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1201
11 dual simplex iterations (0 in phase I)
CPLEX 

CPLEX 12.8.0.0: optimal solution; objective 233.2807971
8 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 226.0162826
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 240.8341888
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 242.1598371
8 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 230.6785367
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 238.0052691
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 251.7599569
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 219.5333617
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 224.2974217
11 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 243.3831373
3 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal sol

CPLEX 12.8.0.0: optimal solution; objective 45612
7 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 47798
7 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 55443
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 43211
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44874
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 52804
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 45274
14 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46663
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 40567
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 45233
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 37721
5 dual simplex iterations (0 in phase 

CPLEX 12.8.0.0: optimal solution; objective 1321.976744
44 dual simplex iterations (27 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1091.522727
38 dual simplex iterations (18 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1068.5
23 dual simplex iterations (13 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1331.69186
35 dual simplex iterations (22 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1150.883333
43 dual simplex iterations (32 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1213.659091
27 dual simplex iterations (13 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1160.840426
36 dual simplex iterations (17 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1255.97561
30 dual simplex iterations (17 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1096.729167
43 dual simplex iterations (22 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 985.9615385
50 dual simplex iterations (22 in phase I)
CPLEX 12.8.0.0: o

CPLEX 12.8.0.0: optimal solution; objective 230.3302736
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 251.1125835
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 254.0936452
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 238.9888283
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 239.1343189
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 249.0234068
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 236.9297141
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 227.0729736
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 250.2728641
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 221.8361556
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 244.0052833
0 simplex iterations (0 

...

CPLEX 12.8.0.0: optimal solution; objective 411913
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 444012
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 405518
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 418961
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 414725
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 430401
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 409620
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 410189
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 441944
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 413513
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 419045
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective

CPLEX 12.8.0.0: optimal solution; objective 2332.145754
144 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2374.728853
156 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2322.91616
186 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2376.646085
149 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2362.570363
158 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2363.879093
170 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2357.460546
187 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2368.631269
161 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2321.754474
168 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2341.776944
167 dual simplex iterations (0 in phase I)
CPLEX 12.8.

CPLEX 12.8.0.0: optimal solution; objective 12592
269 dual simplex iterations (87 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12158
186 dual simplex iterations (23 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13047
201 dual simplex iterations (16 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12699
155 dual simplex iterations (6 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12587
184 dual simplex iterations (20 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12631
173 dual simplex iterations (10 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12792
156 dual simplex iterations (7 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13074
194 dual simplex iterations (26 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12733
168 dual simplex iterations (16 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 12509
194 dual simplex iterations (16 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 13173
180 dual sim

CPLEX 12.8.0.0: optimal solution; objective 400022.25
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 347490
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 376028
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 362017
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 394808.25
749 dual simplex iterations (376 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 356577
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 392842.25
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 367849
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 378767.5
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 365755.75
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 368016.5
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: op

[0.99313182440390935,
 0.98541590186402661,
 0.99870933906060688,
 0.98029146986395388,
 0.99298223496906091,
 0.82373386241281787,
 0.97922106464204361,
 0.61197019039456901,
 1.0,
 1.0,
 1.0,
 1.0,
 0.94282422409447686,
 0.93308313850749247,
 0.97243561496802555,
 0.9603426856617352,
 0.96802503791178651,
 0.87642711807029317,
 0.88664087377745793,
 0.76392143019082459,
 1.0,
 1.0,
 1.0,
 1.0,
 0.99935565821158634,
 0.99734109051264419,
 0.99996152244924896,
 0.99334041975828025,
 0.99926251867255478,
 0.80970412199475694,
 0.99466216186738865,
 0.5788285324126583,
 1.0,
 0.99999999999999967,
 1.0,
 1.0,
 0.99534742293398115,
 0.98872201984141039,
 0.9993540325025505,
 0.98491956490750809,
 0.99486351021084451,
 0.81979741864112121,
 0.98340065620610595,
 0.64395979785149948,
 1.0,
 0.99999999999999978,
 1.0,
 1.0]

In [11]:
TotalBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, TotalBenefit_ijk_sn_2, TotalBenefit_ijk_sn_1))
TotalBenefit_ijk_sn_ratio

[0.99313182440390935,
 0.98541590186402661,
 0.99870933906060688,
 0.98029146986395388,
 0.99298223496906091,
 0.82373386241281787,
 0.97922106464204361,
 0.61197019039456901,
 1.0,
 1.0,
 1.0,
 1.0,
 0.94282422409447686,
 0.93308313850749247,
 0.97243561496802555,
 0.9603426856617352,
 0.96802503791178651,
 0.87642711807029317,
 0.88664087377745793,
 0.76392143019082459,
 1.0,
 1.0,
 1.0,
 1.0,
 0.99935565821158634,
 0.99734109051264419,
 0.99996152244924896,
 0.99334041975828025,
 0.99926251867255478,
 0.80970412199475694,
 0.99466216186738865,
 0.5788285324126583,
 1.0,
 0.99999999999999967,
 1.0,
 1.0,
 0.99534742293398115,
 0.98872201984141039,
 0.9993540325025505,
 0.98491956490750809,
 0.99486351021084451,
 0.81979741864112121,
 0.98340065620610595,
 0.64395979785149948,
 1.0,
 0.99999999999999978,
 1.0,
 1.0]

## Other Function

In [22]:
# Increase the costs of beef and ham

cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

# 再解一次

ampl.solve()
print("New objective value:", totalcost.value())

# 印出其中項目

Buy = ampl.getVariable('Buy')
print("Buy['BEEF'].val = {}".format(Buy['BEEF'].value()))

# Display the dual value of each constraint

diet = ampl.getConstraint('diet')
for nutr in nutrients:
    print("diet['{}'].dual = {}".format(nutr, diet[nutr].dual()))

Increased costs of beef and ham.
CPLEX 12.8.0.0: optimal solution; objective 144.0120033
0 simplex iterations (0 in phase I)
New objective value: 144.01200332502074
Buy['BEEF'].val = 5.226932668329172
diet['A'].dual = 0.0
diet['C'].dual = 0.0
diet['B1'].dual = 0.0
diet['B2'].dual = 0.7999285120532
diet['NA'].dual = -0.007531172069825434
diet['CAL'].dual = 0.0
